Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Learn how to find bottleneck and performance limiters using Nsight tools
- Learn about ways to to extract more parallelism in loops
- Dig deeper into kernels by analyzing it with Nsight Compute


In this section, we will optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenMP offloading. Before we begin, feel free to have a look at the parallel version of the code and inspect it once again. 

[RDF Parallel Code](../../source_code/openmp/SOLUTION/rdf_offload_split.cpp)

[File Reader](../../source_code/openmp/SOLUTION/dcdread.h)

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf SOLUTION/rdf_offload.cpp

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openacc/rdf_offload.qdrep) and open it via the GUI.

## Further Optimization
Currently both our distributed and workshared parallelism comes from the same loop. In other words, in this example, we used teams and distribute to spread parallelism across the full GPU. As a result, all of the parallelism comes from the outer loop. There are multiple ways to increase parallelism: 

- Moving the `Parallel For` to the inner loop
- Collapsing the loops together

In the following sections, we will explore each of these methods.


### Split Teams Distribute from Parallel For

In this method, we distribute outer loop to thread teams and workshare inner loop across threads.The example below shows usage of OpenMP target splitting teams distribute from parallel for. The example code contains two levels of
OpenMP offloading parallelism in a double-nested loop.

```cpp
#pragma omp target teams distribute 
    for( int j = 1; j < N; j++) {
      #pragma parallel for 
      for( int i = 1; i < M; i++ ) {
            <loop code>
    }
}
```

Look at the profiler report from the previous section again. From the timeline, have a close look at the the kernel functions. Checkout the theoretical occupancy. As shown in the example screenshot below, `pair_gpu_182` kernel has the theoretical occupancy of 50%. It clearly shows that the occupancy is a limiting factor. *Occupancy* is a measure of how well the GPU compute resources are being utilized. It is about how much parallelism is running / how much parallelism the hardware could run.

<img src="../images/data_thread.png">

NVIDIA GPUs are comprised of multiple streaming multiprocessors (SMs) where it can manage up to 2048 concurrent threads (not actively running at the same time). Low occupancy shows that there are not enough active threads to fully utilize the computing resources. Higher occupancy implies that the scheduler has more active threads to choose from and hence achieves higher performance. So, what does this mean in OpenACC execution model?



Now, lets start modifying the original code and split the teams distribute. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Inspect the compiler feedback (you should get a similar output as below) you can see from *Line 172* and *Line 178* that it separated the teams distribute from parallel for.

<img src="../images/openmp_feedback_offload_split.png">

Make sure to validate the output by running the executable and validate the output.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_split.qdrep) and open it via the GUI. Compare the execution time for the `Pair_Calculation` from the NVTX row with the previous section and see how much the performance was improved.

Feel free to checkout the [solution](../../source_code/openmp/SOLUTION/rdf_offload_split.cpp) to help you understand better.

## Collapse clause

Specifying the `collapse(n)` clause takes the next `n` tightly-nested loops, folds them into one, and applies the OpenMP directives to the new loop. Collapsing loops means that two loops of trip counts N and M respectively will be automatically turned into a single loop with a trip count of N times M. By collapsing two or more parallel loops into a single loop the compiler has an increased amount of parallelism to use when mapping the code to the device. On highly parallel architectures, such as GPUs, this will give us more parallelism to distribute and better performance.

Try using the collapse clause and observe any performance difference. How much this optimization will speed-up the code will vary according to the application and the target accelerator, but it is not uncommon to see large speed-ups by using collapse on loop nests.

In the example below, we collapse the two loops before applying both teams and thread parallelism to both.

```cpp
#pragma omp target teams distribute parallel for collapse(2) 
for (int i = 0; i < N; i++ )
{
    for (int j=0;j< M;j++)
        < loop code >
} 
```

Now, lets start modifying the original code and add the collapse clause. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Make sure to validate the output by running the executable and validate the output.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_collapse ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_collapse.qdrep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/openmp_gpu_collapse.png">

Compare the execution time for the `Pair_Calculation` from the NVTX row (annotated in Red rectangle in the example screenshot) with the previous section. It is clear the using collapse clause improved the performance by extracting more parallelism. Please note when you compare the two methods explored here, one may give better results than the other.

Feel free to checkout the [solution](../../source_code/openmp/SOLUTION/rdf_offload_collapse.cpp) to help you understand better.

### `num_teams(n)` and `num_threads`

When using the `distribute parallel for` construct, the for loop is distributed across all threads for all teams of the current teams region. For example, if there are 10 teams, and each team consists of 256 threads, the loop will be distributed across 2560 threads. You can explicitly specify the number of threads to be created in the team using the `num_threads(m)`. Moreover, you can specify the maximum number of teams created by using `num_teams(n)` (the actual number of teams may be smalled than this number). You can add these attributes to the `collpase(x)` clause or `parallel for` clause. 


```cpp
#pragma omp target teams distribute parallel for collapse(2) num_threads(256)
for (int i = 0; i < N; i++ )
{
    for (int j=0;j< M;j++)
        < loop code >
} 
```

Now, lets start modifying the original code and add `num_threads(n)` to the `collapse` clause. From the top menu, click on *File*, and *Open* `rdf.cpp` and `dcdread.h` from the current directory at `C/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

**NOTE:** To do some more experiment, try changing the value of `n` to 128, 256, 512. Then compile, run and profile the code and compare the result. Are there any differences? What does the profiler show?

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvc++ -mp=gpu -Minfo=mp -o rdf rdf.cpp 

Make sure to validate the output by running the executable and validate the output.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_num ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_num.qdrep) and open it via the GUI. Feel free to checkout the example [solution](../../source_code/openmp/SOLUTION/rdf_offload_collapse_num.cpp) to help you understand better.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip). Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a></p>

-----


# Links and Resources
[OpenMP Programming Model](https://computing.llnl.gov/tutorials/openMP/)

[OpenMP Target Directive](https://www.openmp.org/wp-content/uploads/openmp-examples-4.5.0.pdf)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0). 